In [1]:
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import cv2
import sys
import os
from torch.utils.data import DataLoader

os.environ["CUDA_VISIBLE_DEVICES"]="4"
sys.path.append("../")

from configs.baseline_effnetb7_3planes_v0_noinfoloss import Configs


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from senet_hoa.callbacks.evaluation import compute_2d_dice

In [3]:
cfg = Configs()

/usr/local/lib/python3.8/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnetv2_m_in21k to current tf_efficientnetv2_m.in21k.
  model = create_fn(
/usr/local/lib/python3.8/dist-packages/torch/storage.py:315: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.
  warnings.warn(message, UserWarning)


length of train: 14800, length of valid: 513


In [4]:

import torch
from tqdm import tqdm
import os
import numpy as np
from scipy import stats
from collections import defaultdict
from senet_hoa.utils.surface_dice_metric import compute_surface_dice_at_tolerance,compute_surface_distances
from senet_hoa.dataset.segment_3d_dataset import LargeImageInferenceCollator

def compute_2d_dice(gt,pred):
    return compute_surface_dice_at_tolerance(compute_surface_distances(gt,pred,(1,1)),0)

class ModelValidationCallback:
    def __init__(self,model,valid_loader,threshold=0.5,device="cpu",output_dir="./"):
        self.model = model
        self.valid_loader = valid_loader
        self.threshold = threshold
        self.device = device
        self.output_dir = output_dir
        self.score =-1

    def __call__(self, current_step):
        # Initializing storage for overall truths and predictions
        ground_truth = []
        predictions = []
        
        for images,coors,shape,mask_gt in tqdm(self.valid_loader):
            with torch.no_grad():
                prediction = self.model(images.to(self.device)).detach().cpu()
                prediction = torch.sigmoid(LargeImageInferenceCollator.combine_masks_into_image(shape,coors,prediction))
            ground_truth.append(mask_gt.squeeze(0).numpy())
            predictions.append(prediction.squeeze(0).squeeze(1).numpy())
        return ground_truth,predictions

In [5]:
cfg.load_state_dict("/app/segment/solve-diseases/workdir/configs.baseline_effnetb7_3planes_v0_noinfoloss/bestmodel_opa.pkl")
cfg.model.to(cfg.device)
cfg.model.eval()
1

loading model checkpoint from epoch:  29600


1

In [6]:
# cfg.train_dataset.slices = [cfg.train_dataset.slices[x] for x in range(0,len(cfg.train_dataset.slices),len(cfg.train_dataset.slices)//500)]

In [18]:
valid_loader = DataLoader(cfg.valid_dataset, batch_size=1, pin_memory=cfg.PIN_MEMORY, num_workers=cfg.NUM_WORKERS_VAL,shuffle=False,collate_fn=cfg.inference_collator)

In [25]:
evaluation_callback = ModelValidationCallback(cfg.model,valid_loader,cfg.PRED_THRESHOLD,cfg.device)

In [26]:
gt,preds = evaluation_callback(0)

100%|██████████| 513/513 [05:02<00:00,  1.70it/s]


In [27]:
from senet_hoa.dataset.transforms import FilterSmallComponents
tform = FilterSmallComponents()

In [28]:
# all_scores = [compute_2d_dice(x>0,y>cfg.PRED_THRESHOLD) for x,y in zip(gt,preds)] 
all_scores = [compute_2d_dice(x>0,tform.apply_to_mask(y>0.5)) for x,y in zip(gt,preds)] 
all_scores = [x for x in all_scores if not np.isnan(x)]
score = np.mean(all_scores) if len(all_scores)>0 else 0
print(f"thres={cfg.PRED_THRESHOLD} : {score}")

/app/segment/solve-diseases/notebooks/../senet_hoa/utils/surface_dice_metric.py:1181: RuntimeWarning: invalid value encountered in double_scalars
  surface_dice = (overlap_gt + overlap_pred) / (np.sum(surfel_areas_gt) +


thres=0.5 : 0.49483603224629713


In [ ]:
8 0.49483603224629713

7 0.49483603224629713

6 0.5045748301923862


In [23]:
# # all_scores = [compute_2d_dice(x>0,tform.apply_to_mask(y>0.5)) for x,y in zip(gt,preds)] 
# all_scores = [x for x in all_scores if not np.isnan(x)]
# score = np.mean(all_scores) if len(all_scores)>0 else 0
# print(f"thres={thres} : {score}")

thres=9 : 0.4600667044033393


# Threshold calculation

In [24]:
for thres in range(4,10):
    all_scores = [compute_2d_dice(x>0,y>thres/10) for x,y in zip(gt,preds)] 
    all_scores = [x for x in all_scores if not np.isnan(x)]
    score = np.mean(all_scores) if len(all_scores)>0 else 0
    print(f"thres={thres} : {score}")